# Artificial Intelligence :  Computer Assignment 2 - Game
> __Morteza Nouri, 810198481__

## Goals:
- Learning Adversarial Search algorithm(minimax)
- How to formulate problems (Abstraction)
- Familiarity with decision making and game theory
- Familiarity with a variety of game strategies (zero-sum, general , ...)

## Description:
> In this project we want to develop an agent who play connect4 game with another player. We use minimax algorithm to find different combinations of possible moves for players (using backtracking) and then choose chain of moves to win. Because the state space of moves can be very large we use heuristic function and don't explore all depths in a decision tree to reach leaves.


### Heuristic Function:

In practice, because of resources limitation  we cannot search the decision tree to leaves. The solution is to use Depth-Limited search and replace terminal utilities with an evaluation
function for non-terminal positions.<br>
Evaluation function score non-terminals and it is typically weighted linear sum of features.<br>
For this game, I use three features: 2-consecutive identical pieces, 3-consecutive identical pieces and 4-consecutive identical pieces, each has its own value. <br>
I travese horizontally, vertically and diagonal to find mentioned features, then for one player I added this feature value to total score while for another player I subtract feature value from total score.


### Part 1: Minimax

In [ ]:
def evaluation(self):
    """
    evaluate score of current board state, this method traverse in 4 direction and find 2-consecutive and 3-consecutive 
    YOU pieces and CPU pieces and then increment the score.

    Outputs
    ----------
    returns the score of current board state
    """
    score = 0
    score += self.__horizontal_score(self.__CONNECT_NUMBER - 2)
    score += self.__horizontal_score(self.__CONNECT_NUMBER - 1)
    score += self.__vertical_score(self.__CONNECT_NUMBER - 2)
    score += self.__vertical_score(self.__CONNECT_NUMBER - 1)
    score += self.__diagonal_up_score(self.__CONNECT_NUMBER - 2)
    score += self.__diagonal_up_score(self.__CONNECT_NUMBER - 1)
    score += self.__diagonal_down_score(self.__CONNECT_NUMBER - 2)
    score += self.__diagonal_down_score(self.__CONNECT_NUMBER - 1)
    return score

In [ ]:
def __horizontal_score(self, consecSlice):
    """
    Traverse horizontally to find consecSlice-consecutive pieces of YOU and CPU and increments for YOU pieces
    and decrements for CPU pieces.

    Inputs
    -------
    consecutive slice to consider in traversing

    Output
    --------
    horizontal evaluation score of board state
    """
    score = 0
    for i in range(self.rows):
        for j in range(self.columns - consecSlice + 1):
            win = self.board[i][j:j+consecSlice]
            if win == [self.YOU] * 3:
                score += 45
            elif win == [self.CPU] * 3:
                score -= 45
            elif win == [self.YOU] * 2:
                score += 4
            elif win == [self.CPU] * 2:
                score -= 4
            else: pass
    return score

def __vertical_score(self, consecSlice):
    score = 0
    board = np.array(self.board)
    for j in range(self.columns):
        col_array = [int(x) for x in list(board[:,j])]
        for i in range(self.rows - consecSlice + 1):
            win = col_array[i:i+consecSlice]
            if win == [self.YOU] * 3:
                score += 45
            elif win == [self.CPU] * 3:
                score -= 45
            elif win == [self.YOU] * 2:
                score += 4
            elif win == [self.CPU] * 2:
                score -= 4
            else:
                pass
    return score 

def __diagonal_up_score(self, consecSlice):
    score = 0
    for r in range(self.rows - consecSlice):
        for c in range(self.columns - consecSlice):
            win = [self.board[r+i][c+i] for i in range(consecSlice)]
            if win == [self.YOU] * 3:
                score += 45
            elif win == [self.CPU] * 3:
                score -= 45
            elif win == [self.YOU] * 2:
                score += 4
            elif win == [self.CPU] * 2:
                score -= 4
            else:
                pass
    return score 

def __diagonal_down_score(self, consecSlice):
    score = 0
    for r in range(self.rows - consecSlice):
        for c in range(self.columns - consecSlice):
            win = [self.board[r+consecSlice-i][c+i] for i in range(consecSlice)]
            if win == [self.YOU] * 3:
                score += 45
            elif win == [self.CPU] * 3:
                score -= 45
            elif win == [self.YOU] * 2:
                score += 4
            elif win == [self.CPU] * 2:
                score -= 4
            else:
                pass
    return score 


In [ ]:

def minimax(self, depth, turn):
    self.explored += 1
    pm = self.get_possible_moves()
    score = 0
    if depth == 0 or self.check_if_player_has_won(self.YOU) or self.check_if_player_has_won(self.CPU) or len(pm) == 0:
        if self.check_if_player_has_won(self.YOU) or self.check_if_player_has_won(self.CPU) or len(pm) == 0:
            if self.check_if_player_has_won(self.YOU):
                score += 1000000
            elif self.check_if_player_has_won(self.CPU):
                score -= 1000000
            else:
                score += self.evaluation()
        else:
            score += self.evaluation()

        return None, score

    boardCpy = copy.deepcopy(self.board)
    if turn == self.YOU:  # Maximizer
        bestValue = float('-inf')
        column = choice(pm)
        for m in pm:
            if self.is_move_valid(m):
                self.register_input(m, self.YOU)
                value = self.minimax(depth-1, self.CPU)[1]
                if value > bestValue:
                    column = m
                    bestValue = value
            self.board = copy.deepcopy(boardCpy)
        return column, bestValue

    else:  # turn == self.CPU, Minimizer
        bestValue = float('inf')
        column = choice(pm)
        for m in pm:
            if self.is_move_valid(m):
                self.register_input(m, self.CPU)
                value = self.minimax(depth-1, self.YOU)[1]
                if value < bestValue:
                    column = m
                    bestValue = value
            self.board = copy.deepcopy(boardCpy)
        return column, bestValue

### Results for Minimax:

| Test | Board | Depth | AvgTime(s) | TotalTime(200 executions)(s)  | WinRate(wins in 200 executions) | ExploredStates(cumulative)|
| :-: | :-: | :-: | :-: | :-: | :-: | :-:|
|1| 6 * 7 | 1 | 0.0147 | 2.9500 | 70% | 32
|2| 6 * 7 | 3 | 0.6759 | 135.186 | 99% | 1800
|3| 6 * 7 | 5 | 31.128 | 6743.650 | 99% | 74000
|4| 7 * 8 | 1 | 0.0197 | 3.945 | 73% | 40
|5| 7 * 8 | 3 | 1.1378 | 277.572 | 99.5% | 2750
|6| 7 * 8 | 5 | 72.433 | 21319.1683 | 100% | 160000
|7| 7 * 10| 1 | 0.0366 | 7.337847 | 80.5% | 44
|8| 7 * 10| 3 | 3.392 | 678.5175 | 98.5% |5400
|9| 7 * 10| 5 | 376.6435 | 75328.7 | 99% |420000

### Part 2: Minimax with Alpha-Beta Pruning
__Alpha__ is the best value that the __maximizer__ currently can guarantee at that level or above. <br>
__Beta__ is the best value that the __minimizer__ currently can guarantee at that level or above.


In [ ]:
def minimaxAlphaBeta(self, depth, turn, alpha = float('-inf'), beta = float('inf')):
    self.explored += 1
    pm = self.get_possible_moves()
    score = 0
    if depth == 0 or self.check_if_player_has_won(self.YOU) or self.check_if_player_has_won(self.CPU) or len(pm) == 0:
        if self.check_if_player_has_won(self.YOU) or self.check_if_player_has_won(self.CPU) or len(pm) == 0:
            if self.check_if_player_has_won(self.YOU):
                score += 1000000
            elif self.check_if_player_has_won(self.CPU):
                score -= 1000000
            else:
                score += self.evaluation()
        else:
            score += self.evaluation()

        return None, score

    boardCpy = copy.deepcopy(self.board)
    if turn == self.YOU:  # Maximizer
        bestValue = float('-inf')
        column = choice(pm)
        for m in pm:
            if self.is_move_valid(m):
                self.register_input(m, self.YOU)
                value = self.minimaxAlphaBeta(depth-1, self.CPU, alpha, beta)[1]
                if value > bestValue:
                    column = m
                    bestValue = value
                alpha = max(alpha, value)
                if alpha >= beta:
                    beta
            self.board = copy.deepcopy(boardCpy)
        return column, bestValue

    else:  # turn == self.CPU, Minimizer
        bestValue = float('inf')
        column = choice(pm)
        for m in pm:
            if self.is_move_valid(m):
                self.register_input(m, self.CPU)
                value = self.minimaxAlphaBeta(depth-1, self.YOU, alpha, beta)[1]
                if value < bestValue:
                    column = m
                    bestValue = value
                beta = min(beta, value)
                if beta <= alpha:
                    break
            self.board = copy.deepcopy(boardCpy)
        return column, bestValue

### Results for Alpha-Beta Pruning:

| Test | Board | Depth | AvgTime(s) | TotalTime(200 executions)(s)  | WinRate(wins in 200 executions) | ExploredStates(cumulative)|
| :-: | :-: | :-: | :-: | :-: | :-: | :-:|
|1| 6 * 7 | 1 | 0.0142 | 2.8493 | 68% | 30 |
|2| 6 * 7 | 3 | 0.28001 | 56.002 | 98.5% | 800 |
|3| 6 * 7 | 5 | 12.6825 | 2536.5 | 99% | 34600 |
|4 | 6 * 7 | 7 | 211.915 | 42383.16 | 99.5% | 845540 |
|5| 7 * 8 | 1 | 0.01965 | 4.345 | 71% | 36 |
|6| 7 * 8 | 3 | 0.56511 | 113.022 | 98% | 1029 |
|7| 7 * 8 | 5 | 14.9017 | 2980.34 | 99% | 64500 |
|8| 7 * 8 | 7 | 243.6938|48738.76 | 99.5%| 580000 |
|9| 7 * 10| 1 | 0.03180 | 6.46721 | 81.5% | 33 |
|10| 7 * 10| 3 | 1.2146 | 243.12| 98.5% | 1500 |
|11| 7 * 10| 5 | 117.7886 | 23557.72 | 98.5% |215700 |
|12| 7 * 10| 7 | 2064.8254 | 412965.08 | 99.5% | 2980000 |

### Part 3: Answer Questions

__1__: The final decision made by Minimax largely depends on how well the heuristic function is. Therefore, designing a reasonable heuristic function is paramount. So a good heuristic function should be informed and admissible. The heuristic function which considers more features is better. For example in my heuristic features that I mentioned above, if there are 3-identical-connected pieces, the probabilty of wining is high, and for 2-connected this probability is lower and etc. <br>
another heuristic can counts all YOU and CPU pieces in the board and returns the difference between the two numbers, but this heuristic is not accurate because consecutive connected pieces are important to us.


__2__:  As depth of searching keeps increasing, the heuristic has better functionality and win rate increases, but resource utilization(time, memory) increases instead, because we explore more states.

__3__: The effectiveness of alpha-beta pruning is highly dependent on the order in which each node is examined. Move order is an important aspect of alpha-beta pruning. In worst case, algorithm doesn't prune any of the leaves of tree. In best case, lots of pruning happen in the tree and best moves occur at left side of tree. In this project I use random order for visiting nodes. but for better performance of alpha-beta pruning we can do some rules to find better ordering:
- Order the nodes in the tree such that the best nodes are checked first.
- We can store the states, as there is a possibility that states may repeat.
- Occur the best move from the shallowest node.
